In [2]:
%matplotlib

Using matplotlib backend: MacOSX


In [3]:

import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
%matplotlib inline
sns.set_palette('colorblind')
sns.set_style('white')

In [4]:
import pandas as pd
import nltk
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [5]:
from nltk.corpus import brown

In [6]:
from collections import Counter

In [7]:
def get_data():
    df = pd.read_csv('newtrain.csv')
    return df

In [8]:
def add_features(df):

    #tokenize text
    pattern = r'''(?x)    # set flag to allow verbose regexps
         ([A-Z]\.)+        # abbreviations, e.g. U.S.A.
       | \w+([-']\w+)*        # words with optional internal hyphens
       | \$?\d+(\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
       | \.\.\.            # ellipsis
       | [.,;"?():-_`]+  # these are separate tokens
     '''

    tokenize = lambda text: nltk.regexp_tokenize(text, pattern)
    df['Tokens'] = df['Text'].apply(tokenize)
    
    def pos_tag(text):
        tuples = nltk.pos_tag(text)
        tags = []
        for t in tuples:
            tags.append(t[1])
        return tags
    def get_nouns(text):
        wnlemmatizer = nltk.WordNetLemmatizer()
        tuples = nltk.pos_tag(text)
        tags = []
        for t in tuples:
            if t[1][0] == 'N':
                tags.append(wnlemmatizer.lemmatize(t[0]))
        return tags
    df['Nouns'] = df['Tokens'].apply(get_nouns)
    df['POS'] = df['Tokens'].apply(pos_tag)
    make_string = lambda a: ' '.join(i for i in a)
    df['Nouns'] = df['Nouns'].apply(make_string)
    df['POS'] = df['POS'].apply(make_string)
    return df

In [23]:
#vectorize data
#turns words into a list of vectors - vector length is the total number of words
#Vector elements correspond to 1 word (1/0 if word is/not present in the current item)
def get_labels_features(df, max_f):
    vec = CountVectorizer(analyzer = 'char_wb', ngram_range= (5,5), stop_words = ENGLISH_STOP_WORDS)
    vec_tar = LabelEncoder()
    targets = vec_tar.fit_transform(df['Category'])
    
    counts = vec.fit_transform(df['Text'])
    print("Counts shape: " + str(counts.shape))
    nouns = vec.fit_transform(df['Nouns'])
    print("Nouns shape: " + str(nouns.shape))
    pos = vec.fit_transform(df['POS'])
    print("POS shape: " + str(pos.shape))
    
    
    tfidf_transformer = TfidfTransformer()
    tfidf = tfidf_transformer.fit_transform(counts)
    
    #To combine or not to combine
    nouns_array = nouns.toarray()
    pos_array = pos.toarray()
    combined = np.hstack((nouns_array, pos_array))
    return targets, counts

In [10]:
#evaluate using 10 fold cross-validation, inspect results
def cross_validate(targets, features):
    #SVC
    mod = LinearSVC(C=.1)
    cv = StratifiedShuffleSplit(targets, n_iter=10, test_size=.1)

    scores = []
    for tr, tt in cv:
        mod.fit(features[tr], targets[tr])
        scores.append(mod.score(features[tt], targets[tt]))

    print('\nLinear SVC\n\t mean score: {0}'.format(np.mean(scores)))
    
    #svm
    mod = SGDClassifier(loss = 'hinge',penalty = 'L2',alpha = 1e-3)
    scores = []
    for tr, tt in cv:
        mod.fit(features[tr], targets[tr])
        scores.append(mod.score(features[tt],targets[tt]))
    print('\nLinear SVM\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    
    #naive bayes
    mod = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
    scores = []
    for tr, tt in cv:
        mod.fit(features[tr], targets[tr])
        scores.append(mod.score(features[tt],targets[tt]))
    print('\nNaive Bayes\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    logreg = LogisticRegression()
    scores = []
    for tr, tt in cv:
        logreg.fit(features[tr], targets[tr])
        scores.append(logreg.score(features[tt],targets[tt]))
    print('\nLogReg\n\t mean score: {0}'.format(np.mean(scores)))
    


In [11]:
df = get_data()
df = add_features(df)
df.head()

,Category,Text,Tokens,Nouns,POS
0,5,why are yawns contagious? when people yawn,"[why, are, yawns, contagious, ?, when, people,...",yawn people,WRB VBP NNS JJ . WRB NNS VBP
1,6,what is trans fat? how to reduce that? i heard...,"[what, is, trans, fat, ?, how, to, reduce, tha...",trans tras body food,WP VBZ NNS JJ . WRB TO VB IN . PRP VBP IN NNS ...
2,1,roth ira vs 401k? what is the difference betwe...,"[roth, ira, vs, 401k, ?, what, is, the, differ...",roth ira v difference roth ira prefer,NN NN NNS CD . WP VBZ DT NN IN NN NN CC CD . W...
3,1,how many planes fedex has? i heard that it is ...,"[how, many, planes, fedex, has, ?, i, heard, t...",plane fedex airline world,WRB JJ NNS NN VBZ . PRP VBP IN PRP VBZ DT JJS ...
4,2,what is the best photo slideshow creation appl...,"[what, is, the, best, photo, slideshow, creati...",photo slideshow creation application photo sli...,WP VBZ DT JJS NN NN NN NN WP VBZ DT JJS NN NN ...


In [24]:
result = get_labels_features(df, 1000)
labels = result[0]
x = result[1]
cross_validate(labels, x)

Counts shape: (2698, 23364)
Nouns shape: (2698, 11808)
POS shape: (2698, 39)


NameError: global name 'TfidfTransformer' is not defined

In [13]:
#result = get_labels_features(df, 1000)
#labels = result[0]
#x = result[1]
#cross_validate(labels, x)

In [14]:
#result = get_labels_features(df, 1000)
#labels = result[0]
#x = result[1]
#cross_validate(labels, x)